In [ ]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping


In [ ]:
from cnn.cnn import load_dataset, create_model, steps
from dataset import dataset, load_files
from utilities.labels import LABELS
from dataset.data_augmentation import augmentated_filenames2

In [ ]:
data_dir = "./data/mit_bih/"
save_dir = "./data/beats_img/"
_dataset_dir = './data/beats_img'
_model = './trained_models/cnn_aami_baseline.h5'

In [ ]:
train = None
validation = None
augmentation = False
_batch_size = 32
_size = (64, 64)
_epochs = 30

In [ ]:
model = create_model(out_dim=4)

train, validation, test = dataset.load_files(_dataset_dir, 
                                             verbose=True, 
                                             keep_ratio=0.1)
    
print(len(train))
print(len(validation))
print(len(test))

In [ ]:
if augmentation:
    train = augmentated_filenames2(train)

callbacks_list = [ModelCheckpoint(_model, monitor='val_loss', save_best_only=False),
                  EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto',
                                baseline=None, restore_best_weights=False),
                  ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, verbose=0, mode='auto')
# TrainValTensorBoard(write_graph=False)
]

data_gen_train = load_dataset(train, _dataset_dir,
                              _batch_size, _size,
                              label_type = "AAMI",
                              random_crop=augmentation,
                              random_rotate=augmentation,
                              flip=augmentation)

data_gen_valid = load_dataset(validation, _dataset_dir,
                              _batch_size, _size,
                              label_type = "AAMI",
                              random_crop=augmentation,
                              random_rotate=augmentation,
                              flip=augmentation)

model.fit_generator(data_gen_train,
                    steps_per_epoch=steps(train, _batch_size),
                    epochs=_epochs,
                    validation_data=data_gen_valid,
                    validation_steps=steps(validation, _batch_size),
                    callbacks=callbacks_list)

In [ ]:
from keras.models import load_model

model = load_model(_model)

In [ ]:
from evaluation import evaluate_cnn

In [ ]:
evaluate_cnn(model, label_type="AAMI", keep_ratio=0.1)
print("")

In [10]:
TEST_DATA_PATH = 'data/beats_img/test.txt'

MITBIH_CLASSES = ['N', 'L', 'R', 'e', 'j', 'A', 'a', 'J', 'S', 'V', 'E', 'F', 'P', '/', 'f', 'u']
MITBIH_CLASSES_REDUCED = ['N', 'L', 'R', 'e', 'j', 'A', 'a', 'J', 'S', 'V', 'E', 'F']  # , 'P', '/', 'f', 'u']

AAMI2MITBIH_MAPPING = {
    "N": ['N', 'L', 'R'],
    "SVEB": ['A', 'a', 'J', 'S', 'e', 'j'],
    "VEB": ['V', 'E'],
    "F": ['F'],
    "Q": ['P', '/', 'f', 'u'],
}

AAMI2MITBIH_MAPPING_REDUCED = {
    "N": ['N', 'L', 'R'],
    "SVEB": ['A', 'a', 'J', 'S', 'e', 'j'],
    "VEB": ['V', 'E'],
    "F": ['F'],
    # "Q": ['P', '/', 'f', 'u'],
}

# AAMI_CLASSES_REDUCED = sorted(AAMI2MITBIH_MAPPING_REDUCED.keys())
AAMI_CLASSES_REDUCED = ['F', 'N', 'SVEB', 'VEB']

MITBIH2TYPE2_MAPPING = {".": "NOR",
                        "N": "NOR",
                        "V": "PVC",
                        "/": "PAB",
                        "L": "LBB",
                        "R": "RBB",
                        "A": "APC",  # APB
                        "a": "aPC",  # aAPB
                        "e": "AEB",
                        "J": "JPB",
                        "j": "JEB",
                        "S": "SVP",  # SVPB
                        "!": "VFW",
                        "F": "FUS",  # Fusion
                        "E": "VEB"}

# TYPE2_CLASSES_REDUCED = sorted(MITBIH2TYPE2_MAPPING.values())
TYPE2_CLASSES = ['AEB',
                 'APC',
                 'FUS',
                 'JEB',
                 'JPB',
                 'LBB',
                 'NOR',
                 'NOR',
                 'PAB',
                 'PVC',
                 'RBB',
                 'SVP',
                 'VEB',
                 'VFW',
                 'aPC']

MITBIH2TYPE2_MAPPING_REDUCED = {".": "NOR",
                                "N": "NOR",
                                "V": "PVC",
                                "/": "PAB",
                                "L": "LBB",
                                "R": "RBB",
                                "A": "APC",
                                "!": "VFW",
                                "E": "VEB"}

# TYPE2_CLASSES_REDUCED = sorted(MITBIH2TYPE2_MAPPING_REDUCED.values())
TYPE2_CLASSES_REDUCED = ['APC', 'LBB', 'NOR', 'NOR', 'PAB', 'PVC', 'RBB', 'VEB', 'VFW']

mitclasses2type2classes = lambda mit_classes: [MITBIH2TYPE2_MAPPING[mit_cls] for mit_cls in mit_classes if mit_cls in MITBIH2TYPE2_MAPPING]
mitclasses2type2classes_reduced = lambda mit_classes: [MITBIH2TYPE2_MAPPING_REDUCED[mit_cls] for mit_cls in mit_classes if mit_cls in MITBIH2TYPE2_MAPPING_REDUCED]
AAMI2TYPE2_MAPPING_REDUCED = dict([(aami_label, mitclasses2type2classes(mitclasses)) for aami_label, mitclasses in AAMI2MITBIH_MAPPING_REDUCED.items()])
AAMI2TYPE2_MAPPING_REDUCED2 = dict([(aami_label, mitclasses2type2classes_reduced(mitclasses)) for aami_label, mitclasses in AAMI2MITBIH_MAPPING_REDUCED.items()])


In [11]:
AAMI2TYPE2_MAPPING_REDUCED

{'N': ['NOR', 'LBB', 'RBB'],
 'SVEB': ['APC', 'aPC', 'JPB', 'SVP', 'AEB', 'JEB'],
 'VEB': ['PVC', 'VEB'],
 'F': ['FUS']}